In [99]:
import numpy as np
import pandas as pd
import random
import copy

time: 642 µs (started: 2022-10-14 13:21:27 +00:00)


In [97]:
!pip install ipython-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 29.7 MB/s 


In [98]:
%load_ext autotime

time: 268 µs (started: 2022-10-14 13:21:22 +00:00)


In [100]:
colnames = ['x', 'y', 'cost']
TSPA = pd.read_csv('TSPA.csv', header=None, sep=';', names=colnames)
TSPB = pd.read_csv('TSPB.csv', header=None, sep=';', names=colnames)

time: 7.36 ms (started: 2022-10-14 13:21:32 +00:00)


In [101]:
TSPA

,x,y,cost
0,1510,353,84
1,3042,123,483
2,2145,389,1462
3,1030,365,1986
4,2125,360,145
...,...,...,...
195,1023,1258,197
196,3192,1271,1183
197,3735,1879,1461
198,3983,299,1309


time: 8.14 ms (started: 2022-10-14 13:21:34 +00:00)


In [102]:
tspa_dic = TSPA.to_dict('index')
tspb_dic = TSPB.to_dict('index')

time: 2.39 ms (started: 2022-10-14 13:21:37 +00:00)


In [103]:
# TODO: maybe change calculate_distances to matrix 200x200 and create it at the beggining

def calculate_distances(node, data):
    distances = {}
    starting_cords = np.array([node['x'], node['y']])
    for key in data.keys():
        ending_cords = np.array([data[key]['x'], data[key]['y']])
        dist = round(np.linalg.norm(starting_cords - ending_cords))
        distances[key] = dist
    return distances

def calculate_single_distance(starting_node, ending_node):
    starting_cords = np.array([starting_node['x'], starting_node['y']])
    ending_cords = np.array([ending_node['x'], ending_node['y']])
    dist = round(np.linalg.norm(starting_cords - ending_cords))
    return dist

time: 2.13 ms (started: 2022-10-14 13:21:41 +00:00)


In [104]:
def random_solution(starting_node, data, total_cost, chosen_nodes):
    chosen_nodes.append(starting_node)
    while len(data) > 101:
        start_node = data.pop(starting_node)
        random_node = random.choice(list(data.keys()))
        end_node = data[random_node]
        dist = calculate_single_distance(start_node, end_node)
        total_cost.append(start_node['cost'])
        total_cost.append(dist)
        random_solution(random_node, data, total_cost, chosen_nodes)
    return sum(total_cost), chosen_nodes

time: 1.48 ms (started: 2022-10-14 13:21:44 +00:00)


In [105]:
def random_solution_iterate(data):
    min_cost = np.inf
    max_cost = 0
    min_cost_nodes = []
    for i in range(len(data)):
        total_cost, chosen_nodes = random_solution(i, copy.deepcopy(data), total_cost = [], chosen_nodes = [])
        if total_cost < min_cost:
            min_cost = total_cost
            min_cost_nodes = chosen_nodes
        if total_cost > max_cost:
            max_cost = total_cost
    print(min_cost)
    print(max_cost)
    print(len(min_cost_nodes))
    print(min_cost_nodes)

random_solution_iterate(tspa_dic)


232917
295334
100
[154, 76, 40, 195, 1, 11, 124, 188, 105, 60, 186, 17, 87, 102, 169, 80, 62, 140, 149, 135, 8, 45, 28, 106, 68, 12, 197, 69, 51, 32, 120, 24, 77, 117, 95, 2, 73, 44, 43, 63, 146, 103, 54, 35, 14, 199, 57, 192, 75, 34, 46, 22, 153, 191, 179, 190, 152, 23, 155, 175, 163, 4, 151, 181, 130, 134, 178, 115, 119, 126, 110, 53, 9, 145, 139, 133, 59, 10, 196, 90, 39, 159, 21, 91, 86, 137, 156, 98, 26, 79, 50, 37, 72, 189, 104, 121, 96, 198, 67, 180]
time: 441 ms (started: 2022-10-14 13:21:48 +00:00)


In [106]:
def nearest_neighbor(starting_node, data, total_cost,chosen_nodes):
    chosen_nodes.append(starting_node)
    while len(data) > 101:
        start_node = data.pop(starting_node)
        distances = calculate_distances(start_node, data)
        key_min = min(distances.keys(), key=(lambda k: distances[k]))
        nearest_node = data[key_min]
        total_cost.append(start_node['cost'])
        total_cost.append(distances[key_min])
        nearest_neighbor(key_min, data, total_cost, chosen_nodes)
    return sum(total_cost), chosen_nodes


time: 4.14 ms (started: 2022-10-14 13:21:54 +00:00)


In [109]:
def nearest_neighbor_iterate(data):
    min_cost = np.inf
    max_cost = 0
    min_cost_nodes = []
    for i in range(len(data)):
        total_cost, chosen_nodes = nearest_neighbor(i, copy.deepcopy(data), total_cost = [], chosen_nodes = [])
        if total_cost < min_cost:
            min_cost = total_cost
            min_cost_nodes = chosen_nodes
        if total_cost > max_cost:
            max_cost = total_cost
    print(min_cost)
    print(max_cost)
    print(len(min_cost_nodes))
    print(min_cost_nodes)

nearest_neighbor_iterate(tspa_dic)

107727
122611
100
[41, 137, 177, 1, 75, 189, 109, 119, 100, 33, 26, 13, 52, 148, 48, 92, 16, 152, 11, 162, 160, 198, 106, 125, 8, 105, 123, 124, 80, 191, 95, 169, 110, 139, 196, 51, 5, 112, 165, 73, 179, 12, 89, 42, 94, 122, 72, 190, 98, 156, 172, 66, 6, 68, 24, 67, 87, 144, 102, 44, 133, 154, 81, 171, 104, 184, 194, 79, 21, 157, 129, 170, 186, 127, 88, 153, 161, 76, 145, 151, 49, 38, 0, 115, 69, 149, 50, 121, 91, 114, 2, 4, 29, 187, 192, 150, 199, 39, 174, 173]
time: 23.1 s (started: 2022-10-14 13:38:38 +00:00)


In [ ]:
def cycle_greedy(starting_node, data, total_cost,chosen_nodes):
    pass